In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
data = pd.read_excel('Sandhi.xlsx')
data

,Sanskrit,Sandhi_split
0,गच्छामि,"गच्छ,अमि"
1,भवार्जुन,"भव,अर्जुन"
2,भीष्मासभरक्षितम्,"भीष्म,असभ,रक्षितम्"
3,पाण्डवाश्चैव,"पाण्डवास्,च,एव"
4,प्रथमोऽध्यायः,"प्रथमस्,अध्यायस"
5,ॐ,ओम
6,ज्ञानयोगेन,"ज्ञान,योगेन"
7,कृष्णैकत्वम्,"कृष्ण,एकत्वम्"
8,विद्यालयः,"विद्या,आलयः"
9,नमसीश्वरम्,"नमसि,ईश्वरम्"


In [3]:
X = data['Sanskrit']
y = data['Sandhi_split']

In [ ]:
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(X)

# Convert characters to sequences of integers
X_seq = tokenizer.texts_to_sequences(X)

# Padding sequences to ensure uniform length
X_padded = pad_sequences(X_seq)

# Label encoding for target
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Building the RNN model
vocab_size = len(tokenizer.word_index) + 1
max_len = X_padded.shape[1]

model = Sequential([
    Embedding(vocab_size, 50, input_length=max_len),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [17]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


1/1 [==============================] - 6s 6s/step - loss: 2.3974 - accuracy: 0.1667 - val_loss: 2.4081 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 55ms/step - loss: 2.3883 - accuracy: 0.3333 - val_loss: 2.4117 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 51ms/step - loss: 2.3812 - accuracy: 0.3333 - val_loss: 2.4158 - val_accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 56ms/step - loss: 2.3689 - accuracy: 0.5000 - val_loss: 2.4209 - val_accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 52ms/step - loss: 2.3633 - accuracy: 0.3333 - val_loss: 2.4272 - val_accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 51ms/step - loss: 2.3506 - accuracy: 0.5000 - val_loss: 2.4340 - val_accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 50ms/step - loss: 2.3424 - accuracy: 0.3333 - val_loss: 2.4421 - val_accuracy: 0.0000e+00

In [18]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

1/1 [==============================] - 0s 96ms/step - loss: 2.4545 - accuracy: 0.0000e+00
Test Accuracy: 0.00


In [19]:
example_text = ["विद्यालयः"]
example_seq = tokenizer.texts_to_sequences(example_text)
example_padded = pad_sequences(example_seq, maxlen=max_len)
prediction = model.predict(example_padded)
predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])
print(f"Prediction for '{example_text[0]}': {predicted_class[0]}")

1/1 [==============================] - 0s 311ms/step
Prediction for 'विद्यालयः': भानु,उदयः
